In [34]:
import pandas as pd

# Create a DataFrame
df = pd.DataFrame()

In [35]:
samples = 1000

In [36]:
import numpy as np


def x_random_numbers():
    x1 = np.random.randint(0, 100)
    x2 = np.random.randint(0, 100)
    while x1 == x2:
        x2 = np.random.randint(0, 100)
    return min(x1, x2), max(x1, x2)


X = []
# Generate random numbers and add them to the DataFrame
for i in range(1000):
    x1, x2 = x_random_numbers()
    X.append([x1, x2])

df['x'] = X

# Display the first few rows of the DataFrame
df.head()

,x
0,"[19, 37]"
1,"[29, 78]"
2,"[46, 67]"
3,"[69, 89]"
4,"[86, 90]"


In [37]:
import random


def generate_numbers(df):
    # Generate 99 numbers between 1 and 99
    positive_number = [random.randint(1, 99) for _ in range(samples)]

    # Generate negative numbers for each generated number
    negative_numbers = [-x for x in positive_number]

    # Create pairs of original number and its negative in a list
    number_pairs = [[neg, pos]
                    for neg, pos in zip(negative_numbers, positive_number)]

    # Add the list of pairs as a new column to the DataFrame
    df['y'] = number_pairs


generate_numbers(df)

In [38]:
# random state column with random values from 0 to 99
def random_state(df):
    df['random_state'] = [random.randint(0, 99) for _ in range(samples)]


random_state(df)
df.head()

,x,y,random_state
0,"[19, 37]","[-41, 41]",27
1,"[29, 78]","[-71, 71]",81
2,"[46, 67]","[-80, 80]",98
3,"[69, 89]","[-14, 14]",20
4,"[86, 90]","[-66, 66]",93


In [39]:
df.to_csv('data.csv', index=False)

In [40]:
import random

odd_numbers = [random.choice(range(3, 14, 2)) for _ in range(samples)]

df['sections'] = odd_numbers

df.head()

,x,y,random_state,sections
0,"[19, 37]","[-41, 41]",27,7
1,"[29, 78]","[-71, 71]",81,9
2,"[46, 67]","[-80, 80]",98,13
3,"[69, 89]","[-14, 14]",20,9
4,"[86, 90]","[-66, 66]",93,7


In [41]:
# Load random matrix csv
random_mat = pd.read_csv('random_matrix.csv')
random_mat.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,88,24,92,29,42,98,76,41,93,9,41,94,60,24
1,41,93,24,42,9,92,41,24,94,98,76,88,29,60
2,29,92,60,94,76,41,41,9,93,24,42,24,98,88
3,93,92,41,42,98,24,60,94,29,24,41,76,88,9
4,92,41,29,76,24,98,42,94,24,88,41,60,93,9


In [42]:
def add_random_mat(df, random_mat):
    def get_random_mat_row(row):
        index = row['random_state'] % 100
        return random_mat.iloc[index].values

    df['random_mat_row'] = df.apply(get_random_mat_row, axis=1)
    return df

In [43]:
add_random_mat(df, random_mat)
df.head()

,x,y,random_state,sections,random_mat_row
0,"[19, 37]","[-41, 41]",27,7,"[93, 92, 98, 42, 24, 9, 41, 76, 60, 94, 29, 24..."
1,"[29, 78]","[-71, 71]",81,9,"[24, 92, 94, 93, 9, 29, 42, 60, 76, 24, 98, 88..."
2,"[46, 67]","[-80, 80]",98,13,"[94, 60, 88, 41, 42, 24, 24, 93, 29, 41, 76, 9..."
3,"[69, 89]","[-14, 14]",20,9,"[98, 29, 88, 41, 60, 94, 42, 93, 24, 92, 76, 2..."
4,"[86, 90]","[-66, 66]",93,7,"[41, 42, 76, 93, 24, 24, 9, 94, 98, 41, 88, 92..."


In [44]:
# Function to generate evenly spaced numbers
def generate_points_y(row):
    return list(np.linspace(row['y'][0], row['y'][1], row['sections']+1))


def generate_points_x(row):
    return list(np.linspace(row['x'][0], row['x'][1], row['sections']+1))


# Apply the function to each row
df['x_points'] = df.apply(generate_points_x, axis=1)
df['y_points'] = df.apply(generate_points_y, axis=1)

In [45]:
df.head()

,x,y,random_state,sections,random_mat_row,x_points,y_points
0,"[19, 37]","[-41, 41]",27,7,"[93, 92, 98, 42, 24, 9, 41, 76, 60, 94, 29, 24...","[19.0, 21.571428571428573, 24.142857142857142,...","[-41.0, -29.285714285714285, -17.5714285714285..."
1,"[29, 78]","[-71, 71]",81,9,"[24, 92, 94, 93, 9, 29, 42, 60, 76, 24, 98, 88...","[29.0, 34.44444444444444, 39.888888888888886, ...","[-71.0, -55.22222222222222, -39.44444444444444..."
2,"[46, 67]","[-80, 80]",98,13,"[94, 60, 88, 41, 42, 24, 24, 93, 29, 41, 76, 9...","[46.0, 47.61538461538461, 49.23076923076923, 5...","[-80.0, -67.6923076923077, -55.38461538461539,..."
3,"[69, 89]","[-14, 14]",20,9,"[98, 29, 88, 41, 60, 94, 42, 93, 24, 92, 76, 2...","[69.0, 71.22222222222223, 73.44444444444444, 7...","[-14.0, -10.88888888888889, -7.777777777777778..."
4,"[86, 90]","[-66, 66]",93,7,"[41, 42, 76, 93, 24, 24, 9, 94, 98, 41, 88, 92...","[86.0, 86.57142857142857, 87.14285714285714, 8...","[-66.0, -47.14285714285714, -28.28571428571428..."


In [46]:
# Create a new column 'rand_vals' by taking the first 'sections'+1 values from 'random_mat_row'
df['rand_vals'] = df.apply(
    lambda row: row['random_mat_row'][:row['sections']+1], axis=1)

In [47]:
df['points'] = df.apply(lambda row: list(
    zip(row['x_points'], row['y_points'])), axis=1)

In [48]:
# Update y values in 'points' column
def update_points(row):
    new_points = []
    for i, (x, y) in enumerate(row['points']):
        if i % 2 == 0:  # subtract for even index
            new_y = y - row['rand_vals'][i]
        else:  # add for odd index
            new_y = y + row['rand_vals'][i]
        new_points.append((x, new_y))
    return new_points


df['poly_points'] = df.apply(update_points, axis=1)

In [ ]:
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial

# Apply Lagrange interpolation on 'poly_points' and save the equation into a new column 'polynomial'


def interpolate_points(row):
    x, y = zip(*row['poly_points'])
    poly = lagrange(x, y)
    return Polynomial(poly).coef.tolist()


df['polynomial'] = df.apply(interpolate_points, axis=1)

In [49]:
df.to_csv('data.csv', index=False)